<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/evaluation_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Perplexity

# Diversity

In [ ]:
!pip install nltk

In [ ]:
from collections import Counter

from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np

In [ ]:
def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        words = seq.split()
        unigrams = Counter(words)
        bigrams = Counter(zip(words, words[1:]))
        # print(unigrams)
        # print(bigrams)

        intra_dist1.append((len(unigrams)+1e-12) / (len(words)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(words)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    # print("\n")
    # print("unigrams all: ", unigrams_all)
    # print("bigrams all: ", bigrams_all)
    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)
    # print("\n")
    # print("intra for each sentence (unigram):", intra_dist1)
    # print("intra for each sentence (bigram):", intra_dist2)
    # print("\n")
    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

# cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
# values = distinct(cand)
# print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")


In [ ]:
cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
values = distinct(cand)
print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")

In [ ]:
def get_list_of_sentences(file_name, label = "    labels:"):
    with open(file_name) as file:
        lines = file.readlines()
    i =0
    dialog_score_pairs = []
    turns = []
    list_diversity_scores = []
    for line in lines:
        if ("NEW EPISODE" in line):
            if len(turns) > 0 : 
                diversity_scores = distinct(turns)
                list_diversity_scores.append(diversity_scores)
                dialog_score_pairs.append([turns, diversity_scores])

            turns = []
        if (line.startswith(label)):
            turns.append(line[len(label)+1:len(line)-3])
    
    average_diversity_score = [
        np.average([item[0] for item in list_diversity_scores]),
        np.average([item[1] for item in list_diversity_scores]),
        np.average([item[2] for item in list_diversity_scores]),
        np.average([item[3] for item in list_diversity_scores])
    ]

    average_of_turns = np.average([len(item[0]) for item in dialog_score_pairs])
    
    return [dialog_score_pairs, average_of_turns, average_diversity_score]

# file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"
# labels = get_list_of_sentences(file_name4, "    labels:")
# values4 = distinct(labels)
# labels

In [ ]:
file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"

labels_n_scores = get_list_of_sentences(file_name4)
print("labels:")
print(f"turns average count: {round(labels_n_scores[1],2)}\n",
    f"intra_distinc1: {labels_n_scores[2][0]}\n", 
    f"intra_distinc2: {labels_n_scores[2][1]}\n", 
    f"inter_distinc1: {labels_n_scores[2][2]}\n", 
    f"inter_distinc2: {labels_n_scores[2][3]}\n")


generated_sentences4_n_scores = get_list_of_sentences(file_name4, "     model:")
# print(f"number of generated sentences in 4th model: {len(generated_sentences4)}")

print("4:")
print(f"turns average count: {round(generated_sentences4_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences4_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences4_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences4_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences4_n_scores[2][3]}\n")

file_name5 = "models-examples\\05-finetuned-beam_prmts_infrnc-testset-output.txt"
generated_sentences5_n_scores = get_list_of_sentences(file_name5, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("5:")
print(f"turns average count: {round(generated_sentences5_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences5_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences5_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences5_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences5_n_scores[2][3]}\n")

file_name9 = "models-examples\\09-finetuned-400m-topk-5epochs-testset-output.txt"
generated_sentences9_n_scores = get_list_of_sentences(file_name9, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("9:")
print(f"turns average count: {round(generated_sentences9_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences9_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences9_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences9_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences9_n_scores[2][3]}\n")

# Coherency

In [ ]:
!git lfs install
!git clone https://huggingface.co/Sahajtomar/french_semantic

!pip install sentence-transformers

In [4]:
output_lines = []
with open("/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt") as f:
    output_lines = f.readlines()
print(len(output_lines))
    
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Sahajtomar/french_semantic')

12121


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/933 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# for pure datasets
for i,line in enumerate(output_lines):
    if (line.startswith("[id]: dataset_french")): break
        
label = ""
generated = ""
text = ""
context = ""
list_of_similarity_scores_for_turns_in_a_dialog = []
list_of_similarity_scores_for_all_dialogs = []

for i,line in enumerate(output_lines[i+1:]):
    if (line.startswith("[labels]: ")):
        label = line[len("[labels]: ")+1:].replace("\n","")
#         print("context:", context)
#         print("label:", label)
        
        # print(label, context)
        embeddings_label = model.encode(label, convert_to_tensor=True)
        embeddings_context = model.encode(context[:300], convert_to_tensor=True)
        cosine_similarity = util.pytorch_cos_sim(embeddings_label, embeddings_context)
#         print("similarity:", cosine_similarity.item(),"\n\n")
        list_of_similarity_scores_for_turns_in_a_dialog.append(cosine_similarity.item())
        context += label
        
#     elif (line.startswith("     model:")):
#         generated = line[len("     model:")+1:].replace("\n","")
        
    elif (line.startswith("[text]: ")):
        text = line[len("[text]: ")+1:].replace("\n","")
        context += text
        
    elif (line.startswith("- - - - - - - END OF EPISODE - - - - - - - - - -")):
        average_similarity_score = sum(list_of_similarity_scores_for_turns_in_a_dialog) / len(list_of_similarity_scores_for_turns_in_a_dialog)
        list_of_similarity_scores_for_all_dialogs.append(average_similarity_score)
        context = ""
        label = ""
        print(f"{i}: Similarity Score for a dialog:",average_similarity_score)
    
#     if len(list_of_similarity_scores_for_all_dialogs) > 5: break


print("Count: ", len(list_of_similarity_scores_for_all_dialogs))
print("Average: ", sum(list_of_similarity_scores_for_all_dialogs)/ len(list_of_similarity_scores_for_all_dialogs))
print(list_of_similarity_scores_for_all_dialogs)